In [ ]:
# Import pustaka yang diperlukan dari LangChain dan pustaka Python standar
from langchain_community.chat_models import ChatOllama  # Model LLM dari Ollama
from langchain_core.prompts import ChatPromptTemplate  # Template prompt untuk LLM
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Split teks menjadi chunks
from langchain_community.vectorstores import FAISS  # Vector Store menggunakan FAISS
from langchain_community.embeddings import OllamaEmbeddings  # Embeddings dari Ollama
from langchain_core.runnables import RunnablePassthrough  # Untuk menjalankan pipeline
from langchain.document_loaders import TextLoader  # Loader untuk file .txt
from langchain.document_loaders import PyPDFLoader  # Loader untuk file .pdf
import os  # Pustaka standar Python untuk operasi file

# 1. Fungsi untuk Mengunggah dan Membaca Dokumen
def load_document(file_path):
    # Mengecek ekstensi file untuk menentukan loader yang sesuai
    if file_path.endswith(".txt"):
        loader = TextLoader(file_path)  # Menggunakan TextLoader untuk file .txt
    elif file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)  # Menggunakan PyPDFLoader untuk file .pdf
    else:
        # Jika format file tidak didukung, raise error
        raise ValueError("Format file tidak didukung. Gunakan file .txt atau .pdf.")
    
    # Memuat dokumen dan mengembalikan dalam bentuk list of LangChain Documents
    documents = loader.load()
    return documents

# 2. Fungsi untuk Split Dokumen menjadi Chunks
def split_document(documents):
    # Inisialisasi RecursiveCharacterTextSplitter untuk membagi dokumen menjadi chunks
    # chunk_size=500 artinya setiap chunk maksimal 500 karakter
    # chunk_overlap=50 artinya setiap chunk memiliki overlap 50 karakter dengan chunk sebelumnya
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    
    # Membagi dokumen menjadi chunks
    texts = text_splitter.split_documents(documents)
    return texts

# 3. Fungsi untuk Membuat Vector Store
def create_vector_store(texts):
    # Membuat embeddings menggunakan model Llama3.2
    embeddings = OllamaEmbeddings(model="llama3.2:1b-instruct-fp16")
    
    # Membuat vector store menggunakan FAISS dari dokumen yang sudah di-embed
    vectorstore = FAISS.from_documents(texts, embeddings)
    return vectorstore

# 4. Fungsi untuk Meringkas Dokumen
def summarize_document(vectorstore, query):
    # Inisialisasi LLM menggunakan model Llama3.2
    llm = ChatOllama(model="llama3.2:1b-instruct-fp16", temperature=0)
    
    # Membuat template prompt untuk ringkasan dokumen
    prompt_template = ChatPromptTemplate.from_template(
        "Buat ringkasan lengkap dan terstruktur dari teks berikut. Ringkasan harus mencakup semua poin penting dan disajikan dengan rapi:\n\n{context}\n\nRingkasan:"
    )
    
    # Membuat retriever dari vector store untuk mendapatkan chunk yang relevan
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  
    # k=5 artinya mengambil 5 chunk paling relevan
    
    # Membuat RAG (Retrieval-Augmented Generation) Chain
    # Data dari retriever dipasangkan dengan prompt template dan dijalankan di LLM
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt_template
        | llm
    )
    
    # Menjalankan RAG Chain dengan query yang diberikan
    response = rag_chain.invoke(query)
    
    # Mengembalikan hasil ringkasan
    return response.content

# Path ke file PDF atau TXT
file_path = "artikel1.pdf"  # Ganti dengan path file Anda

# Memuat dokumen
try:
    print("Memuat dokumen...")
    documents = load_document(file_path)  # Memanggil fungsi untuk memuat dokumen
    print("Dokumen berhasil dimuat.")
except Exception as e:
    # Jika terjadi error saat membaca file, tampilkan pesan error
    print(f"Error saat membaca file: {str(e)}")
    raise

# Membagi dokumen menjadi chunks
print("Membagi dokumen menjadi chunks...")
texts = split_document(documents)  # Memanggil fungsi untuk split dokumen

# Membuat vector store
print("Membuat vector store...")
vectorstore = create_vector_store(texts)  # Membuat vector store dari chunks yang telah di-embed

# Meringkas dokumen
print("Meringkas dokumen...")
query = "Buat ringkasan lengkap dan terstruktur dari dokumen ini."
summary = summarize_document(vectorstore, query)  # Menggunakan LLM untuk meringkas dokumen

# Tampilkan hasil ringkasan
print("\nRingkasan Dokumen:")
print(summary)


Memuat dokumen...
Dokumen berhasil dimuat.
Membagi dokumen menjadi chunks...
Membuat vector store...
Meringkas dokumen...

Ringkasan Dokumen:
Berikut adalah ringkasan lengkap dan terstruktur dari teks yang Anda berikan:

**Teks**

Merupakan langkah yang digunakan untuk pengujian sistem penjualan barang di Swalayan Brastagi Medan. Dalam pre-procesing data, salah satu langkah yang dilakukan adalah transformasi setiap nilai atribut yang sama ke bentuk numerik sehingga mudah dilakukan untuk proses pemecahan masalah dan bentukan data dan sampelnya.

**Pre-Procesing Data**

1. Nama barang: nama barang yang akan dijadikan acuan untuk dimasukan kedalam daftar stok barang yang dibutuhkan.
2. Penjualan pada Swalayan Brastagi Medan: Journal of Computing and Informatics Research, Vol 1, No 1, November 2021.

**Pola Belanja Pembeli**

Sistem yang berjalan pada Swalayan Brastagi Medan dalam hal Pola Belanja Pembeli menggunakan sebuah sistem pencatatan dengan menggunakan media komputer. Dimana pola b